In [1]:
import pandas as pd
import requests

In [2]:
# API: default is 1000
limit = '$limit=100000000'

### FHV

https://data.cityofnewyork.us/Transportation/2019-For-Hire-Vehicles-Trip-Data/u6nh-b56h

In [3]:
# base url
base_url_fhv = 'https://data.cityofnewyork.us/resource/u6nh-b56h.json'

In [4]:
# filter for March and April
soql_filter_fhv = '?$where=date_extract_m(pickup_datetime) in (3, 4)'

In [5]:
# combine base url, filter, and limit
final_url_fhv = base_url_fhv + soql_filter_fhv + '&' + limit
print(final_url_fhv)

https://data.cityofnewyork.us/resource/u6nh-b56h.json?$where=date_extract_m(pickup_datetime) in (3, 4)&$limit=100000000


In [6]:
# retrieve the data 
request_fhv = requests.get(url=final_url_fhv)
df_fhv = pd.DataFrame(request_fhv.json())

In [7]:
df_fhv.head()

,dispatching_base_num,pickup_datetime,dropoff_datetime,pulocationid,dolocationid
0,B00310,2019-03-06T09:37:12.000,2019-03-06T10:02:37.000,264,265
1,B00310,2019-03-06T09:32:18.000,2019-03-06T09:36:39.000,264,265
2,B00310,2019-03-06T09:45:08.000,2019-03-06T10:08:35.000,264,265
3,B00310,2019-03-06T09:15:28.000,2019-03-06T09:34:32.000,264,265
4,B00310,2019-03-06T09:43:28.000,2019-03-06T09:48:26.000,264,265


In [8]:
df_fhv.shape

(3237176, 5)

In [9]:
df_fhv.to_csv('raw_fhv_ly.csv')

### High-Volume FHV 

https://data.cityofnewyork.us/Transportation/2019-High-Volume-FHV-Trip-Records/4p5c-cbgn

In [10]:
# base url
base_url_hvfhv = 'https://data.cityofnewyork.us/resource/4p5c-cbgn.json'

In [11]:
# filter for March and April
soql_filter_hvfhv = '?$where=date_extract_m(pickup_datetime) in (3, 4)'

In [12]:
# combine base url, filter, and limit
final_url_hvfhv = base_url_hvfhv + soql_filter_hvfhv + '&' + limit
print(final_url_hvfhv)

https://data.cityofnewyork.us/resource/4p5c-cbgn.json?$where=date_extract_m(pickup_datetime) in (3, 4)&$limit=100000000


In [14]:
# retrieve the data 
request_hvfhv = requests.get(url=final_url_hvfhv)
df_hvfhv = pd.DataFrame(request_hvfhv.json())

In [15]:
df_hvfhv.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,pulocationid,dolocationid,sr_flag
0,HV0003,B02889,2019-04-26T19:30:03.000,2019-04-26T19:50:13.000,92,135,1
1,HV0003,B02889,2019-04-26T19:46:07.000,2019-04-26T20:16:35.000,121,95,1
2,HV0003,B02875,2019-04-26T19:57:09.000,2019-04-26T20:06:48.000,43,239,NaN
3,HV0003,B02864,2019-04-26T19:15:26.000,2019-04-26T19:21:47.000,235,94,NaN
4,HV0003,B02864,2019-04-26T19:34:21.000,2019-04-26T19:49:33.000,235,18,NaN


In [16]:
df_hvfhv.shape

(45599365, 7)

In [17]:
df_hvfhv.to_csv('raw_hvfhv_ly.csv')

### Data Cleaning

#### hvhfs_license_num (High-Volume FHV)

The TLC License number of the HVFHS base or business 
- HV0002: Juno
- HV0003: Uber
- HV0004: Via
- HV0005:Lyft

In [18]:
df_hvfhv['hvfhs_license_num'].unique()

array(['HV0004', 'HV0005', 'HV0003', 'HV0002'], dtype=object)

In [19]:
df_hvfhv.groupby(['hvfhs_license_num']).agg({'hvfhs_license_num': 'count'})

,hvfhs_license_num
hvfhs_license_num,
HV0002,1575747
HV0003,32905739
HV0004,2128145
HV0005,8989734


#### sr_flag (High-Volume FHV)

Indicates if the trip was a part of a shared ride chain offered by a High Volume FHV company (e.g. Uber Pool, Lyft Line). For shared trips, the value is 1. For non-shared rides, this field is null.

NOTE: For most High Volume FHV companies, only shared rides that were requested AND matched to another shared-ride request over the course of the journey are flagged. However, Lyft (hvfhs_license_num=’HV0005’) also flags rides for which a shared ride was requested but another passenger was not successfully matched to share the trip—therefore, trips records with SR_Flag=1 from those two bases could indicate EITHER a trip in a shared trip chain OR a trip for which a shared ride was requested but never matched. Users should anticipate an overcount of successfully shared trips completed by Lyft. 

Note also that Juno does not offer shared trips

In [20]:
df_hvfhv['sr_flag'].unique()

array(['1', nan], dtype=object)

In [21]:
df_hvfhv.groupby(['sr_flag']).agg({'sr_flag': 'count'})

,sr_flag
sr_flag,
1,9140841


#### Merge

In [22]:
# define function to look up company name
def company(x):
    if x == 'HV0002':
        return 'Juno'
    elif x == 'HV0003':
        return 'Uber'
    elif x == 'HV0004':
        return 'Via'
    elif x == 'HV0005':
        return 'Lyft'
    else:
        return null

In [23]:
# add fhv type
df_fhv['type'] = 'fhv'
df_hvfhv['type'] = df_hvfhv['hvfhs_license_num'].apply(company)

In [24]:
df_fhv_base = df_fhv[['pickup_datetime', 'dropoff_datetime', 'pulocationid', 'dolocationid', 'type']]
df_fhv_base.head()

,pickup_datetime,dropoff_datetime,pulocationid,dolocationid,type
0,2019-03-06T09:37:12.000,2019-03-06T10:02:37.000,264,265,fhv
1,2019-03-06T09:32:18.000,2019-03-06T09:36:39.000,264,265,fhv
2,2019-03-06T09:45:08.000,2019-03-06T10:08:35.000,264,265,fhv
3,2019-03-06T09:15:28.000,2019-03-06T09:34:32.000,264,265,fhv
4,2019-03-06T09:43:28.000,2019-03-06T09:48:26.000,264,265,fhv


In [25]:
df_hvfhv_base = df_hvfhv[['pickup_datetime', 'dropoff_datetime', 'pulocationid', 'dolocationid', 'type']]
df_hvfhv_base.head()

,pickup_datetime,dropoff_datetime,pulocationid,dolocationid,type
0,2019-03-01T00:13:55.000,2019-03-01T00:28:51.000,36,80,Via
1,2019-03-01T00:23:58.000,2019-03-01T00:43:03.000,37,232,Via
2,2019-03-01T00:03:37.000,2019-03-01T00:15:09.000,25,62,Lyft
3,2019-03-01T00:29:46.000,2019-03-01T00:50:43.000,65,262,Uber
4,2019-03-01T00:58:56.000,2019-03-01T01:20:47.000,140,196,Uber


In [26]:
df_fhvall = pd.concat([df_fhv_base, df_hvfhv_base], axis=0)
df_fhvall.head()

,pickup_datetime,dropoff_datetime,pulocationid,dolocationid,type
0,2019-03-06T09:37:12.000,2019-03-06T10:02:37.000,264,265,fhv
1,2019-03-06T09:32:18.000,2019-03-06T09:36:39.000,264,265,fhv
2,2019-03-06T09:45:08.000,2019-03-06T10:08:35.000,264,265,fhv
3,2019-03-06T09:15:28.000,2019-03-06T09:34:32.000,264,265,fhv
4,2019-03-06T09:43:28.000,2019-03-06T09:48:26.000,264,265,fhv


In [27]:
df_fhvall.shape

(48836541, 5)

In [29]:
# avoid OutOfBoundsDatetime error
df_fhvall['pickup_datetime'] = pd.to_datetime(df_fhvall['pickup_datetime'], errors='coerce')
df_fhvall['dropoff_datettime'] = pd.to_datetime(df_fhvall['dropoff_datetime'], errors='coerce')

In [30]:
df_fhvall['pu_year'] = df_fhvall['pickup_datetime'].dt.year
df_fhvall['pu_month'] = df_fhvall['pickup_datetime'].dt.month
df_fhvall['pu_day'] = df_fhvall['pickup_datetime'].dt.day
df_fhvall['pu_hour'] = df_fhvall['pickup_datetime'].dt.hour

In [32]:
df_fhvall = df_fhvall[df_fhvall['pu_year'] == 2019] # remove error records

In [33]:
df_fhvall.head()

,pickup_datetime,dropoff_datetime,pulocationid,dolocationid,type,dropoff_datettime,pu_year,pu_month,pu_day,pu_hour
0,2019-03-06 09:37:12,2019-03-06T10:02:37.000,264,265,fhv,2019-03-06 10:02:37,2019,3,6,9
1,2019-03-06 09:32:18,2019-03-06T09:36:39.000,264,265,fhv,2019-03-06 09:36:39,2019,3,6,9
2,2019-03-06 09:45:08,2019-03-06T10:08:35.000,264,265,fhv,2019-03-06 10:08:35,2019,3,6,9
3,2019-03-06 09:15:28,2019-03-06T09:34:32.000,264,265,fhv,2019-03-06 09:34:32,2019,3,6,9
4,2019-03-06 09:43:28,2019-03-06T09:48:26.000,264,265,fhv,2019-03-06 09:48:26,2019,3,6,9


### Trip Count

In [34]:
df_agg_pu = df_fhvall.groupby(['type', 'pu_month', 'pu_day', 'pu_hour', 'pulocationid']).agg({'pickup_datetime': 'count'}).rename(columns={'pickup_datetime': 'trips'})
df_agg_pu.head()

trips
type pu_month pu_day pu_hour pulocationid       
Juno 3        1      0       10                5
                             100               8
                             102               2
                             106               4
                             107              11

In [35]:
df_agg_do = df_fhvall.groupby(['type', 'pu_month', 'pu_day', 'pu_hour', 'dolocationid']).agg({'dropoff_datetime': 'count'}).rename(columns={'dropoff_datetime': 'trips'})
df_agg_do.head()

trips
type pu_month pu_day pu_hour dolocationid       
Juno 3        1      0       10                3
                             101               1
                             102               4
                             107               7
                             108               1

In [36]:
df_agg_pu.shape

(1288191, 1)

In [37]:
df_agg_do.shape

(1311920, 1)

In [38]:
df_agg_pu.to_csv('fhv_pickup_ly.csv')

In [39]:
df_agg_do.to_csv('fhv_dropoff_ly.csv')